In [1]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

/home/user05/.pyenv/versions/nlpqaconda/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')


bnpp_newsroom-v1.1.csv already downloaded

bert_qa.joblib already downloaded


### Visualize the dataset

In [4]:
df = pd.read_csv('./data/bert_dataset_prepro.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()
  

,Unnamed: 0,title,postdate,location_name,paragraphs
0,1,"카카오프렌즈순면아동가운-기념타올,답례품,컴퓨터자수",2020-10-05 23:09:00,컴퓨터자수 무료 대구 타월공장 대가야 타월,[판매타올 대가야 12시간 전 이웃추가 본문 기타 기능 본문 폰트 크기 조정 본문...
1,2,(내공100)20대신용+체크카드추천부탁드립니다!!,NaN,네이버지식인,[27살 남자구요 살면서 체크카드만 사용해봤고 주위에 좀 물어보니 신용카드 체크카드...
2,4,[천안직산맛집/음식점/혼밥]직산김치찌개를찾는당신'맛고을묵은지'로오라.,2020-10-05 23:09:00,기록의 일상; 여백의 미,[혼밥 가넝한 쩝쩝박사 12시간 전 이웃추가 본문 기타 기능 본문 폰트 크기 조정...
3,5,방문은내일가겠습니다......,2020-10-05 23:09:00,새파란 하늘,[일상_전체 아틸라 12시간 전 이웃추가 본문 기타 기능 본문 폰트 크기 조정 본...
4,7,창원왁싱엔네일샵에간이샤워시설의조립식샤워부스설치,2020-10-05 23:09:00,에치비1522-5310고객센터,[오늘 설치한 조립식샤워부스 소개합니다창원에 왁싱네일샵에싸이즈 1000×900짜리 ...


In [ ]:
df2 = pd.read_csv('./data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs':literal_eval})
df2 = filter_paragraphs(df2)
df2.head()

In [5]:
cdqa_pipeline = QAPipeline(reader='bert_qa_hwalim_epoch5.joblib', retriever='tfidf', retrieve_by_doc=True)
cdqa_pipeline.fit_retriever(df=df.iloc[:10000,:])
cdqa_pipeline.cuda()

/home/user05/.pyenv/versions/nlpqaconda/lib/python3.7/site-packages/cdqa/pipeline/cdqa_sklearn.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: " ".join(x)


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=False, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=3e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=5,
                         output_dir='models', predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=12,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=True,
           retriever=TfidfRetriever(lowercase=True, max_df=0.85, min_df=2,
                                    ngram_range=(1, 2), preprocessor=None,
                                    stop_words='english',
                                    token_

In [6]:
query = '체크카드 추천 부탁드립니다'
cdqa_pipeline.predict(query)

('되는대로',
 '[나눈질문1]이정도경사오를수있는전동킥보드추천해주세요',
 '되는대로 캡쳐해와서 사진상으로는 잘 판단이 안되실 수 있지만체감상 경사가 30은 되는거 같아요 ㅠ버스 기다리는 시간도 아깝고 출근지가 애매한 자리에 있어서좀 더 효율적으로 움직이고자 탈 것을 구매하려고 하는데요전동킥보드로 사진상의 경사를 오를 수 있나요제 키는 188 몸무게는 92 정돕니다가방무게 포함 97정도오를 수 있는게 있으면 추천 부탁드리고전동킥보드 말고 다른 좋은 제품 알고계시면 추천 부탁드립니다',
 5.194071070036557)

In [ ]:
# text, title, paragraph, final_score